In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Getting started

Welcome in the getting started guide of *PedPy*, we want to guide you through first steps to set up an analysis of a pedestrian experiment.
In this guide you will learn how to load the movement data, setup measurement areas, and finally compute the flow and the densities. 

First things first, to use *PedPy* install it via:

```bash
pip install pedpy
```

Now, you are ready to set up your first analysis with *PedPy*.

## Let's analyze an experiment

This is a bottleneck experiment conducted at the University of Wuppertal in 2018.
You can see the basic setup of the experiment in the picture below:

```{eval-rst}
.. figure:: 040_c_56_h-_cropped.png
    :width: 400px
    :align: center
```

The data for this experiment is available [here](https://doi.org/10.34735/ped.2018.1) and is part of the publication ["Crowds in front of bottlenecks at entrances from the perspective of physics and social psychology"](https://doi.org/10.1098/rsif.2019.0871).
For our analysis we are interested in the flow at the bottleneck and density a short distance in front of the bottleneck.


## Importing pedestrian movement data

The pedestrian movement data in *PedPy* is called trajectory data.

*PedPy* works with {class}`trajectory data <pedpy.data.trajectory_data.TrajectoryData>` which can be created from an import function for specific data files alternatively from a [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) with the following columns:
 
- "ID": unique numeric identifier for each person
- "frame": index of video frame where the positions were extracted
- "X", "Y": position of the person (in meter) 

*PedPy* provides an import function to load trajectory data provided [here](https://ped.fz-juelich.de/da/doku.php).
Since the data we want to analyze is from there, we can directly load the trajectory data with *PedPy*:  

In [ ]:
from pedpy import load_trajectory
import pathlib

traj = load_trajectory(trajectory_file=pathlib.Path("040_c_56_h-.txt"))
traj

The loaded trajectories look like:


In [ ]:
from pedpy import plot_trajectories

plot_trajectories(
    traj=traj,
    traj_color="lightblue",
    traj_alpha=0.5,
    traj_width=1,
    hole_color="lightgrey",
).set_aspect("equal")

## Let's analyze the flow at the bottleneck

For this analysis we need to define a line where we want to measure the flow (see {class}`~pedpy.data.geometry.MeasurementLine`).
We place the line at the bottleneck as below:


In [ ]:
from pedpy import MeasurementLine

measurement_line = MeasurementLine([(0.25, 0), (-0.25, 0)])

In [ ]:
from pedpy import plot_measurement_setup

ax = plot_measurement_setup(
    traj=traj,
    traj_color="lightblue",
    traj_alpha=0.5,
    traj_width=1,
    measurement_lines=[measurement_line],
    ml_color="r",
    ml_width=2,
).set_aspect("equal")

The flow calculation is done with {func}`~pedpy.methods.flow_calculator.compute_n_t`, with our trajectory data, the measurement line ,and the frame rate the data was recorded.

In [ ]:
from pedpy import compute_n_t

nt, _ = compute_n_t(
    traj_data=traj,
    measurement_line=measurement_line,
)
nt.plot(x="Time [s]");

## Let's compute the density in front of the bottleneck

Now we are computing the density in an area in front of the bottleneck.
The density is usually not measured directly at the bottleneck, but a short distance in front as here the highest densities occur.

For this we need to define a {class}`measurement area<pedpy.data.geometry.MeasurementArea>`, which has to be a convex polygon.
Such a measurement area can be created with: 

In [ ]:
from pedpy import MeasurementArea

measurement_area = MeasurementArea(
    [(-0.4, 0.5), (0.4, 0.5), (0.4, 1.3), (-0.4, 1.3)]
)

In [ ]:
from pedpy import plot_measurement_setup

plot_measurement_setup(
    traj=traj,
    traj_color="lightblue",
    traj_alpha=0.5,
    traj_width=1,
    measurement_areas=[measurement_area],
    ma_color="r",
    ma_line_color="r",
    ma_line_width=1,
    ma_alpha=0.2,
).set_aspect("equal")

The density over time is now computed by {func}`~pedpy.methods.density_calculator.compute_classic_density`, which takes the trajectory data and measurement area as inputs:

In [ ]:
from pedpy import compute_classic_density

classic_density = compute_classic_density(
    traj_data=traj, measurement_area=measurement_area
)
classic_density.reset_index().plot.line(x="frame", y="classic density");

## Now use the Voronoi method for computing the density

For this we now need information about the area where pedestrian can move, this is called a {class}`walkable area<pedpy.data.geometry.WalkableArea>` in *PedPy* and is essentially a 2D polygon.
We extract the walkable area from this diagram:

```{eval-rst}
.. figure:: experimental_setup.png
    :width: 400px
    :align: center
```

The resulting polygon is described in the code below:


In [ ]:
from pedpy import WalkableArea

walkable_area = WalkableArea(
    # complete area
    [
        (3.5, -2),
        (3.5, 8),
        (-3.5, 8),
        (-3.5, -2),
    ],
    obstacles=[
        # left barrier
        [
            (-2.8, 7.0),
            (-2.8, 0.0),
            (-0.25, 0.0),
            (-0.25, -1.0),
            (-0.45, -1.0),
            (-0.45, -0.2),
            (-3.0, -0.2),
            (-3.0, 7.0),
            (-2.8, 7.0),
        ],
        # right barrier
        [
            (0.25, -1.0),
            (0.25, 0.0),
            (2.8, 0.0),
            (2.8, 7.0),
            (3.0, 7.0),
            (3.0, -0.2),
            (0.45, -0.2),
            (0.45, -1.0),
            (0.25, -1.0),
        ],
    ],
)

The resulting walkable area will look like this, together with the trajectories and the measurement area:

In [ ]:
from pedpy import plot_measurement_setup

plot_measurement_setup(
    walkable_area=walkable_area,
    hole_color="lightgrey",
    traj=traj,
    traj_color="lightblue",
    traj_alpha=0.5,
    traj_width=1,
    measurement_areas=[measurement_area],
    ma_color="r",
    ma_line_color="r",
    ma_line_width=1,
    ma_alpha=0.2,
).set_aspect("equal")

This time we compute the individual density with {func}`~pedpy.methods.method_utils.compute_individual_voronoi_polygons` using the trajectory data and walkable area as inputs.

In [ ]:
from pedpy import compute_individual_voronoi_polygons

individual = compute_individual_voronoi_polygons(
    traj_data=traj, walkable_area=walkable_area
)

The computed Voronoi polygons at one frame look like:

In [ ]:
from pedpy import plot_voronoi_cells

data = individual.merge(traj.data, on=["ID", "frame"])

plot_voronoi_cells(
    data=data[data.frame == 500],
    walkable_area=walkable_area,
    color_mode="density",
    vmin=0,
    vmax=10,
    show_ped_positions=True,
    ped_size=5,
).set_aspect("equal")

Using this we can then can compute the mean Voronoi density in the previously defined measurement area with {func}`~pedpy.methods.density_calculator.compute_voronoi_density` which takes the individual Voronoi data and the measurement area as inputs:

In [ ]:
from pedpy import compute_voronoi_density

density_voronoi, intersecting = compute_voronoi_density(
    individual_voronoi_data=individual, measurement_area=measurement_area
)
density_voronoi.reset_index().plot.line(x="frame", y="voronoi density");

Now you have set up a first analysis which computed the flow, classic density and Voronoi density in a bottleneck experiment.
With this knowledge you can now start your own analysis.
If you need further information take a look at our [user guide](../user_guide/usage), where you can get more insight what *PedPy* can do for you.

We would love to hear some [feedback](https://github.com/PedestrianDynamics/PedPy/discussions) from you!